# PyCity Schools Analysis

- Your analysis here

---

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()
# school_data_complete.info()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
#"nunique"- number of unique values
school_count = school_data_complete['school_name'].nunique()
print(f"Total number of schools: {school_count}")

Total number of schools: 15


In [3]:
# Calculate the total number of students
student_count = len(school_data_complete['student_name'])
print(f"Total number of students: {student_count:,}")

Total number of students: 39,170


In [4]:
# Calculate the total budget
# to do this we need to drop duplicate school entries to ensure each school is counted only once
total_budget = round(school_data_complete.drop_duplicates(subset='school_name')['budget'].sum(),3)
print(f"Total budget for schools is: ${total_budget:,}")

Total budget for schools is: $24,649,428


In [8]:
# Calculate the average (mean) math score
#average_math_score = (school_data_complete['math_score'].sum()/ student_count)
average_math_score = round(school_data_complete['math_score'].mean(),3)
print(f"Average math score for high schools: {average_math_score}")

Average math score for high schools: 78.985


In [9]:
# Calculate the average (mean) reading score
average_reading_score = round(school_data_complete['reading_score'].mean(),3)
print(f"Average reading score for high schools: {average_reading_score}")

Average reading score for high schools: 81.878


In [10]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = round(passing_math_count / float(student_count) * 100,3)
print(f"Percentage of students who passed math: {passing_math_percentage}%")

Percentage of students who passed math: 74.981%


In [11]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = round(passing_reading_count / float(student_count) * 100,3)
print(f"Percentage of students who passed reading: {passing_reading_percentage}%")

Percentage of students who passed reading: 85.805%


In [12]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = round(passing_math_reading_count /  float(student_count) * 100,3)
print(f"Total students who passed math and reading: {passing_math_reading_count:,}")

print(f"Percentage of students who passed math & reading: {overall_passing_rate}%")

Total students who passed math and reading: 25,528
Percentage of students who passed math & reading: 65.172%


In [13]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing Rate": [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985,81.878,74.981,85.805,65.172


## School Summary

In [14]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [15]:
# Calculate the total student count per school from school_data
# two ways of counting it : other via column 'size' or by counting schools in the school list into combined tables
# per_school_counts = school_data.groupby('school_name')['size'].sum()
per_school_counts = school_data_complete['school_name'].value_counts()
print(f"Total students per each school:\n{per_school_counts}")


Total students per each school:
school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64


In [16]:
# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data_complete.groupby('school_name')['budget'].first()
# sorted_per_school_budget = per_school_budget.sort_values(ascending=False)
print(f"Budget for each school:\n{per_school_budget}")
#per each student 
per_school_capita = per_school_budget / school_data.set_index('school_name')['size']
# sorted_per_school_capita = per_school_capita.sort_values(ascending=False)
print(f"Per capita spending for each school (sorted):\n{per_school_capita}")

# per_school_budget
# per_school_capita

Budget for each school:
school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64
Per capita spending for each school (sorted):
school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High 

In [17]:
# Calculate the average test scores per school from school_data_complete
per_school_math = round(school_data_complete.groupby(['school_name'])['math_score'].mean(),3)
print(f"Average math score per school:\n{per_school_math}")
per_school_reading = round(school_data_complete.groupby(['school_name'])['reading_score'].mean(),3)
print(f"Average reading score per school:\n{per_school_reading}")

Average math score per school:
school_name
Bailey High School       77.048
Cabrera High School      83.062
Figueroa High School     76.712
Ford High School         77.103
Griffin High School      83.351
Hernandez High School    77.290
Holden High School       83.803
Huang High School        76.629
Johnson High School      77.072
Pena High School         83.840
Rodriguez High School    76.843
Shelton High School      83.359
Thomas High School       83.418
Wilson High School       83.274
Wright High School       83.682
Name: math_score, dtype: float64
Average reading score per school:
school_name
Bailey High School       81.034
Cabrera High School      83.976
Figueroa High School     81.158
Ford High School         80.746
Griffin High School      83.817
Hernandez High School    80.934
Holden High School       83.815
Huang High School        81.183
Johnson High School      80.966
Pena High School         84.045
Rodriguez High School    80.745
Shelton High School      83.726
Thomas High Sc

In [18]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
students_passing_math = school_data_complete[school_data_complete['math_score'] >= 70]
school_students_passing_math = students_passing_math.groupby('school_name')['student_name'].count()
print(f"Number of students per school with math scores of 70 or higher:\n{school_students_passing_math}")


Number of students per school with math scores of 70 or higher:
school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: student_name, dtype: int64


In [19]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70]
school_students_passing_reading = students_passing_reading.groupby('school_name')['student_name'].count()
print(f"Number of students per school with reading scores of 70 or higher:\n{school_students_passing_reading}")


Number of students per school with reading scores of 70 or higher:
school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: student_name, dtype: int64


In [20]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]  
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
print(f"Number of students per school with math and reading scores of 70 or higher:\n{school_students_passing_math_and_reading}")


Number of students per school with math and reading scores of 70 or higher:
school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64


In [21]:
# Use the provided code to calculate the passing rates
per_school_passing_math = round(school_students_passing_math / per_school_counts * 100,3)
per_school_passing_reading = round(school_students_passing_reading / per_school_counts * 100,3)
overall_passing_rate = round(school_students_passing_math_and_reading / per_school_counts * 100,3)
overall_passing_rate 

school_name
Bailey High School       54.642
Cabrera High School      91.335
Figueroa High School     53.204
Ford High School         54.290
Griffin High School      90.599
Hernandez High School    53.528
Holden High School       89.227
Huang High School        53.514
Johnson High School      53.539
Pena High School         90.541
Rodriguez High School    52.988
Shelton High School      89.892
Thomas High School       90.948
Wilson High School       90.583
Wright High School       90.333
dtype: float64

In [22]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539


## Highest-Performing Schools (by % Overall Passing)

In [23]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing Rate", ascending=False)  

print("Top 5 Highest-Performing Schools:")
top_schools.index.name = "School Name"
top_schools.head(5)

Top 5 Highest-Performing Schools:


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418,83.849,93.272,97.309,90.948
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274,83.989,93.868,96.540,90.583
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541


## Bottom Performing Schools (By % Overall Passing)

In [24]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values("% Overall Passing Rate", ascending=True)  

print("Bottom 5 Lowest-Performing Schools")
bottom_schools.index.name = "School Name"
bottom_schools.head(5)


Bottom 5 Lowest-Performing Schools


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.843,80.745,66.367,80.220,52.988
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539


## Math Scores by Grade

In [25]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_graders_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_graders_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_graders_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_graders_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    "9th Grade": ninth_graders_math_scores,
    "10th Grade": tenth_graders_math_scores,
    "11th Grade": eleventh_graders_math_scores,
    "12th Grade": twelfth_graders_math_scores
})

# Minor data wrangling
math_scores_by_grade.index.name = "School Name"
# math_scores_by_grade.index.name = None

# Display the DataFrame
print("Math Scores by Grade:")
print_data_math_scores_by_grade = round(math_scores_by_grade, 3).sort_values(by='9th Grade', ascending=False) # for better outlook in DF
print_data_math_scores_by_grade


Math Scores by Grade:


,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Holden High School,83.787,83.430,85.000,82.855
Pena High School,83.625,83.372,84.328,84.122
Thomas High School,83.590,83.088,83.499,83.497
Shelton High School,83.421,82.917,83.383,83.779
Wright High School,83.265,84.010,83.837,83.645
Cabrera High School,83.095,83.155,82.766,83.277
Wilson High School,83.086,83.724,83.195,83.036
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187


## Reading Score by Grade 

In [26]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_graders_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_graders_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_graders_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_graders_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th Grade": ninth_graders_reading_scores,
    "10th Grade": tenth_graders_reading_scores,
    "11th Grade": eleventh_graders_reading_scores,
    "12th Grade": twelfth_graders_reading_scores
})

# Minor data wrangling

# reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]  #it was in the original file, but I do not need this
reading_scores_by_grade.index.name = "School Name"

# Display the DataFrame
print("Reading Scores by Grade:")
print_data_reading_scores_by_grade = round(reading_scores_by_grade, 3).sort_values(by='9th Grade', ascending=False) # for better outlook in DF
print_data_reading_scores_by_grade


Reading Scores by Grade:


,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Shelton High School,84.123,83.442,84.374,82.782
Wilson High School,83.940,84.021,83.765,84.318
Wright High School,83.833,83.813,84.156,84.073
Pena High School,83.807,83.612,84.336,84.591
Thomas High School,83.729,84.254,83.586,83.831
Holden High School,83.677,83.325,83.816,84.699
Cabrera High School,83.676,84.253,83.788,84.288
Griffin High School,83.369,83.707,84.288,84.014
Bailey High School,81.303,80.907,80.946,80.912


## Scores by School Spending

In [27]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]


In [28]:
# Create a copy of the school summary for later aggregations
school_spending_df = per_school_summary.copy()


In [29]:
# Use `pd.cut` on the per_school_capita Series from earlier to categorize per student spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels)

# Convert Spending Ranges (Per Student) to a string
school_spending_df["Spending Ranges (Per Student)"] = school_spending_df["Spending Ranges (Per Student)"].astype(str)
school_spending_df
school_spending_df.index.name = "School Name"
# print(school_spending_df.head())

In [30]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing Rate"].mean()


In [31]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
        "Average Math Score": spending_math_scores,
        "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing Rate": overall_passing_spending
})
      

# Display results
print_data_spending_summary = round(spending_summary, 3) # for better outlook in DF
print_data_spending_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
$585-630,81.900,83.156,87.134,92.718,81.418
$630-645,78.519,81.624,73.484,84.392,62.858
$645-680,76.997,81.028,66.165,81.134,53.527
<$585,83.455,83.934,93.460,96.611,90.370


## Scores by School Size

In [32]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [33]:
# Create a copy of the school summary for later aggregations
school_size_df = per_school_summary.copy()

In [34]:
# Use `pd.cut` on the per_school_counts Series from earlier to categorize school size based on the bins.
school_size_df["School Size"] = pd.cut(per_school_summary["Total Students"],size_bins,labels=labels)

# Convert School Size to a string
school_size_df["School Size"] = school_size_df["School Size"].astype(str)
school_size_df.index.name = "School Name"
school_size_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,School Size
School Name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539,Large (2000-5000)


In [35]:
# Calculate averages for the desired columns.
size_math_scores = school_size_df.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_size_df.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = school_size_df.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["School Size"])["% Overall Passing Rate"].mean()


In [36]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing Rate": size_overall_passing
})

# Display results
print_data_size_summary = round(size_summary,3)
print_data_size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Large (2000-5000),77.746,81.344,69.964,82.767,58.286
Medium (1000-2000),83.374,83.865,93.599,96.791,90.621
Small (<1000),83.822,83.930,93.550,96.100,89.884


In [37]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing Rate"].mean()


## Scores by School Type

In [38]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing Rate": average_percent_overall_passing_by_type
})
    

# Display results
print_data_type_summary = round(type_summary,3)
print_data_type_summary 


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.474,83.896,93.621,96.587,90.432
District,76.957,80.967,66.549,80.799,53.672
